Nesse notebook foi realizado a análise de expressões faciais em imagens estáticas, para a identificação das emoções das pessoas das imagens

In [2]:
import glob
import keras
import random
import cv2 as cv
import numpy as np
import pandas as pd
from warnings import simplefilter

In [35]:
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.utils import np_utils
from keras.layers import Activation
from keras.models import Sequential
from keras.constraints import maxnorm
from keras.utils import to_categorical
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.normalization import BatchNormalization

Removendo Warnings

In [3]:
simplefilter(action='ignore', category=FutureWarning)

#### Labels

In [4]:
emotions = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise']

Carregando os dados 

In [5]:
def get_files(emotion):
    imagens = glob.glob('./emotions/{}/*'.format(emotion))
    random.shuffle(imagens)
    train = imagens[:int(len(imagens) * 0.7)]
    test = imagens[-int(len(imagens) * 0.3):]
    return train, test

Separando os dados de treino e de teste

In [26]:
def make_sets():
    X_train = []
    y_train = []
    X_test = []
    y_test = []

    for emotion in emotions:

        train, test = get_files(emotion)

        for item in train:
            image = cv.imread(item)
            #gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
            X_train.append(image)
            y_train.append(emotions.index(emotion))

        for item in test:
            image = cv.imread(item)
            #gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
            X_test.append(image)
            y_test.append(emotions.index(emotion))

    
    return (np.array(X_train), np.array(y_train),
           np.array(X_test), np.array(y_test))
    #return (np.array(X_train).reshape(156, 350**2), np.array(y_train),
    #       np.array(X_test).reshape(65, 350**2), np.array(y_test))

### Treinando um classificador

In [27]:
X_train, y_train, X_test, y_test = make_sets()

In [8]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("Resultado da Análise: {:.2f}%".format(accuracy_score(y_test, y_pred)*100))

Resultado da Análise: 35.38%


In [31]:
batch_size  = 32 
num_classes = 6
epochz      = 100

In [36]:
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [37]:
def base_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(32,(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3,3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    
    return model

In [38]:
cnn_n = base_model()
cnn_n.summary()
cnn = cnn_n.fit(X_train, y_train,
                batch_size=batch_size,
                epochs=epochz, 
                validation_data=(X_test, y_test), 
                shuffle=True)

scores = cnn_n.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 350, 350, 32)      896       
_________________________________________________________________
activation_12 (Activation)   (None, 350, 350, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 350, 350, 32)      128       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 348, 348, 32)      9248      
_________________________________________________________________
activation_13 (Activation)   (None, 348, 348, 32)      0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 348, 348, 32)      128       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 174, 174, 32)     

KeyboardInterrupt: 